In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#############################################################################STO_averaging--------------------------------------
file_path = "/content/samp(PbI2)_02.xlsx"
sheet1_df = pd.read_excel(file_path, sheet_name="Sheet 1")

num_datasets = 6
signal_amp_columns = [f"Signal-Amp_{i}" for i in range(1, num_datasets + 1)]
time = "Time-ps_1"
new_df = sheet1_df.loc[:, [time] + signal_amp_columns]
new_df["Signal-Amp_averaged"] = new_df[signal_amp_columns].mean(axis=1)

df_Sam = new_df.loc[:, [time, "Signal-Amp_averaged"]]

#save the averaged data frame - df_Sam
sam_output_file = "/content/data_PbI2_001_(05_02-2025).dat"
df_Sam.to_csv(sam_output_file, sep="\t", header=False, index=False)

#############################################################################SUB_averaging-----------------------------------------

file_path = "/content/ref(quartz).xlsx"
sheet1_df = pd.read_excel(file_path, sheet_name="Sheet 1")

num_datasets = 5
signal_amp_columns = [f"Signal-Amp_{i}" for i in range(1, num_datasets + 1)]
time = "Time-ps_1"
new_df = sheet1_df.loc[:, [time] + signal_amp_columns]

new_df["Signal-Amp_averaged"] = new_df[signal_amp_columns].mean(axis=1)

df_SUB = new_df.loc[:, [time, "Signal-Amp_averaged"]]

#save the averaged data frame - df_SUB
sub_output_file = "/content/data_Quartz_001_(05_02-2025).dat"
df_SUB.to_csv(sub_output_file, sep="\t", header=False, index=False)

###########################################################################-FFT---------------------------------------------------------

def zDfft(time, intensity, npts=0):
    # Convert time from ps to seconds if necessary
    if max(time) < 1:  # Assuming time is in seconds
        time = np.array(time) * 1e12
        intensity = np.array(intensity) * 1e5

    # Determine time step (in seconds)
    deltat = time[1] - time[0]

    # Zero padding or truncation
    if npts > len(time):
        padded_intensity = np.pad(intensity, (0, npts - len(intensity)), mode='constant')
    elif npts > 0:
        padded_intensity = intensity[:npts]
    else:
        padded_intensity = intensity

    # FFT parameters
    numpts = 2 * len(padded_intensity)
    f = (1 / deltat) * np.arange(numpts // 2) / numpts  # Frequency array in THz

    # Perform FFT
    spectrum = np.fft.fft(padded_intensity, numpts) / 100
    spectrum = spectrum[:numpts // 2]  # Take the first half (positive frequencies)

    # Normalize the spectrum
    s = spectrum * deltat / (5 * 6.6727e-3)  # Normalization factor

    # Set DC component to zero
    s[0] = 0

    return f, s

# Step 4: Load the averaged data for both STO and SUB
# Reloading the saved data for STO
loaded_data_sam = pd.read_csv(sam_output_file, sep='\t', header=None, names=['time', 'Signal-Amp_averaged'])
time_ps_sam = loaded_data_sam['time'].values
intensity_sam = loaded_data_sam['Signal-Amp_averaged'].values

# Reloading the saved data for SUB
loaded_data_sub = pd.read_csv(sub_output_file, sep='\t', header=None, names=['time', 'Signal-Amp_averaged'])
time_ps_sub = loaded_data_sub['time'].values
intensity_sub = loaded_data_sub['Signal-Amp_averaged'].values

# Perform FFT for both STO and SUB
f1, s1 = zDfft(time_ps_sam, intensity_sam, npts=2048)  # STO FFT
f2, s2 = zDfft(time_ps_sub, intensity_sub, npts=2048)  # SUB FFT


fs = 0  # Example starting index for frequency range
fe = len(f1)  # Example ending index for frequency range

# Find indices for the frequency range (for example, 0.3 THz to 1.0 THz)
fsn = np.where(f1 < 0.3)[0]
fs = fsn[-1]  # Adjust fs to the last index below 0.3 THz

fen = np.where(f1 < 3.2)[0]
fe = fen[-1]  # Adjust fe to the last index below 1.0 THz

# Calculate the ratio and unwrap the phase
ratio = s1[fs:fe] / s2[fs:fe]
AT = np.abs(ratio)  # Amplitude
PT = np.unwrap(np.angle(ratio))  # Phase

# Step 6: Save the results (Frequency, Amplitude, Phase) to a file
BBB = np.column_stack([f1[fs:fe], AT, PT])
output_file_result = 'Normalised_PbI2_without cavity'
np.savetxt(output_file_result, BBB, delimiter='\t', header='Frequency (THz)\tAmplitude\tPhase', fmt='%.6e')

print(f"Results saved to {output_file_result}")

# Step 7: Plot the Amplitude Spectrum (AT)
plt.figure(figsize=(6, 5))
plt.plot(f1[fs:fe], AT,)
plt.xlabel('Frequency (THz)', fontsize=15, fontweight='bold', fontstyle='italic')
plt.ylabel('Amplitude', fontsize=15, fontweight='bold', fontstyle='italic')
plt.title('Normalised E-field of PbI2 without cavity', fontsize=15, fontweight='bold', fontstyle='italic')
plt.ylim(0, 1)
plt.legend()
plt.xticks(fontsize=15, fontweight='bold')
plt.yticks(fontsize=15, fontweight='bold')
plt.xlim(0,2.5)
plt.grid(False)
plt.show()